In [9]:

import os, sys, glob, shutil, json
import cv2

from PIL import Image
import numpy as np
from tqdm import tqdm, tqdm_notebook
from visdom import Visdom

%pylab inline
import torch
import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data.dataset import Dataset
class SVHNDataset(Dataset):
    def __init__(self, img_path, img_label, transform=None):
        self.img_path = img_path
        self.img_label = img_label
        if transform is not None:
            self.transform = transform
        else:
            self.transform = None

    def __getitem__(self, index):
        img = Image.open(self.img_path[index]).convert('RGB')

        if self.transform is not None:
            img = self.transform(img)

        # 原始SVHN中类别10为数字0
        lbl = np.array(self.img_label[index], dtype=np.int)
        # print(lbl)
        lbl = list(lbl)  + (5 - len(lbl)) * [10]
        return img, torch.from_numpy(np.array(lbl[:5]))

    def __len__(self):
        return len(self.img_path)

train_path = glob.glob('./input/train/*.png')
train_path.sort()
# print(train_path)
train_json = json.load(open('./input/train.json'))
train_label = [train_json[x]['label'] for x in train_json]
print(len(train_path), len(train_label))
# 验证集

val_path = glob.glob('./input/val/*.png')
val_path.sort()
# print(train_path)
val_json = json.load(open('./input/mchar_val.json'))
val_label = [val_json[x]['label'] for x in val_json]
print(len(val_path), len(val_label))


train_data = SVHNDataset(train_path, train_label,
          transforms.Compose([
              # 缩放到固定尺寸
              transforms.Resize((64, 128)),

              # 随机颜色变换
              transforms.ColorJitter(0.3, 0.3, 0.2),

              # 加入随机旋转
              transforms.RandomRotation(15),
              # 加入随机裁剪
              transforms.RandomCrop((60,120)),
              # 将图片转换为pytorch 的tesntor
              transforms.ToTensor(),

              # 对图像像素进行归一化
              transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
            ]))
val_data = SVHNDataset(val_path, val_label,
          transforms.Compose([
              # 缩放到固定尺寸
              transforms.Resize((64, 128)),

              # 随机颜色变换
              transforms.ColorJitter(0.3, 0.3, 0.2),
              # 加入随机裁剪
              transforms.RandomCrop((60,120)),
              # 加入随机旋转
              transforms.RandomRotation(15),

              # 将图片转换为pytorch 的tesntor
              transforms.ToTensor(),

              # 对图像像素进行归一化
              transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
            ]))
#使用DataLoader封装数据
train_loader = torch.utils.data.DataLoader(
       train_data,
    batch_size=10, # 每批样本个数
    shuffle=False, # 是否打乱顺序
    # num_workers=10, # 读取的线程个数
)
val_loader = torch.utils.data.DataLoader(
       val_data,
    batch_size=10, # 每批样本个数
    shuffle=False, # 是否打乱顺序
    # num_workers=10, # 读取的线程个数
)

Populating the interactive namespace from numpy and matplotlib
30000 30000
10000 10000


# 定义分类模型
这里使用ResNet50的模型进行特征提取

In [10]:
class SVHN_Model1(nn.Module):
    def __init__(self):
        super(SVHN_Model1, self).__init__()

        model_conv = models.resnet50(pretrained=True)
        model_conv.avgpool = nn.AdaptiveAvgPool2d(1)
        model_conv = nn.Sequential(*list(model_conv.children())[:-1])# 去除最后一个fc layer
        self.cnn = model_conv
        self.fc11 = nn.Linear(2048, 128)
        self.fc21 = nn.Linear(2048, 128)
        self.fc31 = nn.Linear(2048, 128)
        self.fc41 = nn.Linear(2048, 128)
        self.fc51 = nn.Linear(2048, 128)
        # self.fc6 = nn.Linear(512, 11)
        self.dropout_1 = nn.Dropout(0.3)
        self.dropout_2 = nn.Dropout(0.3)
        self.dropout_3 = nn.Dropout(0.3)
        self.dropout_4 = nn.Dropout(0.3)
        self.dropout_5 = nn.Dropout(0.3)
        self.fc12 = nn.Linear(128,11)
        self.fc22 = nn.Linear(128,11)
        self.fc32 = nn.Linear(128,11)
        self.fc42 = nn.Linear(128,11)
        self.fc52 = nn.Linear(128,11)
    def forward(self, img):
        feat = self.cnn(img)
        # print(feat.shape)
        feat = feat.view(feat.shape[0], -1)
        c1 = self.fc11(feat)
        c2 = self.fc21(feat)
        c3 = self.fc31(feat)
        c4 = self.fc41(feat)
        c5 = self.fc51(feat)
        c1 = self.dropout_1(c1)
        c2 = self.dropout_2(c2)
        c3 = self.dropout_3(c3)
        c4 = self.dropout_4(c4)
        c5 = self.dropout_5(c5)
        c1 = self.fc12(c1)
        c2 = self.fc22(c2)
        c3 = self.fc32(c3)
        c4 = self.fc42(c4)
        c5 = self.fc52(c5)
        # c6 = self.fc6(feat)
        return c1, c2, c3, c4, c5# , c6

In [11]:
use_cuda = True
def train(train_loader, model, criterion, optimizer, epoch, global_step):
    # 切换模型为训练模式
    model.train()
    train_loss = []

    for data in train_loader:
        if use_cuda:
            pic = data[0].cuda()
            lables = data[1].long().cuda()
        else:
            pic = data[0]
            lables = data[1].long()
        c0, c1, c2, c3, c4 = model(pic)
        loss = criterion(c0, lables[:, 0]) + \
                criterion(c1, lables[:, 1]) + \
                criterion(c2, lables[:, 2]) + \
                criterion(c3, lables[:, 3]) + \
                criterion(c4, lables[:, 4]) # + \
                # criterion(c5, lables[:, 5])
        loss /= 5
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        global_step += 1
        if global_step%100 ==0:
            viz.line([loss.item()], [global_step/100], win='train', update='append')
        train_loss.append(loss.item())
    return np.mean(train_loss)

In [12]:
def validate(val_loader, model, criterion,global_step_val):
    # 切换模型为预测模型
    model.eval()
    val_loss = []

    # 不记录模型梯度信息
    with torch.no_grad():
        for data in val_loader:
            if use_cuda:
                pic = data[0].cuda()
                lables = data[1].long().cuda()
            else:
                pic = data[0]
                lables = data[1].long()
            c0, c1, c2, c3, c4 = model(pic)
            loss = criterion(c0, lables[:, 0]) + \
                    criterion(c1, lables[:, 1]) + \
                    criterion(c2, lables[:, 2]) + \
                    criterion(c3, lables[:, 3]) + \
                    criterion(c4, lables[:, 4]) # + \
                    # criterion(c5, lables[:, 5])
            loss /= 5
            global_step_val += 1
            if global_step_val%100 ==0:
                viz.line([loss.item()], [global_step_val%100], win='val', update='append')
            val_loss.append(loss.item())
    return np.mean(val_loss)

In [13]:
def predict(test_loader, model, tta=10):
    model.eval()
    test_pred_tta = None

    # TTA 次数
    for _ in range(tta):
        test_pred = []

        with torch.no_grad():
            for i, (input, target) in enumerate(test_loader):
                if use_cuda:
                    input = input.cuda()

                c0, c1, c2, c3, c4 = model(input)
                if use_cuda:
                    output = np.concatenate([
                        c0.data.cpu().numpy(),
                        c1.data.cpu().numpy(),
                        c2.data.cpu().numpy(),
                        c3.data.cpu().numpy(),
                        c4.data.cpu().numpy(),
                        # c5.data.cpu().numpy()
                    ], axis=1)
                else:
                    output = np.concatenate([
                        c0.data.numpy(),
                        c1.data.numpy(),
                        c2.data.numpy(),
                        c3.data.numpy(),
                        c4.data.numpy(),
                        # c5.data.numpy()
                    ], axis=1)

                test_pred.append(output)

        test_pred = np.vstack(test_pred)
        if test_pred_tta is None:
            test_pred_tta = test_pred
        else:
            test_pred_tta += test_pred

    return test_pred_tta

# 训练与验证

In [14]:
model = SVHN_Model1()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), 0.001)
best_loss = 1000.0
viz = Visdom()
viz.line([0.], [0.], win='train',opts=dict(title='train loss.',
                                                   legend=['loss']))
viz.line([0.], [0.], win='val',opts=dict(title='val loss.',
                                                   legend=['loss']))
use_cuda = True
if use_cuda:
    model = model.cuda()

for epoch in range(40):
    global_step = 3000 * epoch
    global_step_val = 1000 * epoch
    train_loss = train(train_loader, model, criterion, optimizer, epoch,global_step)
    val_loss = validate(val_loader, model, criterion,global_step_val)

    val_label = [''.join(map(str, x)) for x in val_loader.dataset.img_label]
    val_predict_label = predict(val_loader, model, 1)
    val_predict_label = np.vstack([
        val_predict_label[:, :11].argmax(1),
        val_predict_label[:, 11:22].argmax(1),
        val_predict_label[:, 22:33].argmax(1),
        val_predict_label[:, 33:44].argmax(1),
        val_predict_label[:, 44:55].argmax(1),
        # val_predict_label[:, 55:66].argmax(1),
    ]).T
    val_label_pred = []
    for x in val_predict_label:
        val_label_pred.append(''.join(map(str, x[x!=10])))

    val_char_acc = np.mean(np.array(val_label_pred) == np.array(val_label))

    print('Epoch: {0}, Train loss: {1} \t Val loss: {2}'.format(epoch, train_loss, val_loss))
    print('Val Acc', val_char_acc)
# 记录下验证集精度
    if val_loss < best_loss:
        best_loss = val_loss
        # print('Find better model in Epoch {0}, saving model.'.format(epoch))
        torch.save(model.state_dict(), './output/model.pt')

Setting up a new session...


Epoch: 0, Train loss: 1.2127219400405884 	 Val loss: 1.1555243408083915
Val Acc 0.0239
Epoch: 1, Train loss: 1.1628801651795706 	 Val loss: 1.0511201461553574
Val Acc 0.0222
Epoch: 2, Train loss: 1.0816214495102565 	 Val loss: 1.0264122661352157
Val Acc 0.0911
Epoch: 3, Train loss: 0.8450580192307631 	 Val loss: 0.8446146462261677
Val Acc 0.2157
Epoch: 4, Train loss: 0.6459364933893085 	 Val loss: 0.7690737415999174
Val Acc 0.2842
Epoch: 5, Train loss: 0.5451873091633122 	 Val loss: 0.7232405154705047
Val Acc 0.3366
Epoch: 6, Train loss: 0.48524810664479934 	 Val loss: 0.694256915435195
Val Acc 0.36
Epoch: 7, Train loss: 0.443655635582904 	 Val loss: 0.6381269014701247
Val Acc 0.4087
Epoch: 8, Train loss: 0.4093269582192103 	 Val loss: 0.6366196622997522
Val Acc 0.4147
Epoch: 9, Train loss: 0.3854486995960275 	 Val loss: 0.5991426887959241
Val Acc 0.4439
Epoch: 10, Train loss: 0.36362812971447905 	 Val loss: 0.6603146039396525
Val Acc 0.4207
Epoch: 11, Train loss: 0.3458922279539208 	 

In [17]:
test_path = glob.glob('./input/test/*.png')
test_path.sort()
test_label = [[1]] * len(test_path)
print(len(test_path), len(test_label))

test_loader = torch.utils.data.DataLoader(
    SVHNDataset(test_path, test_label,
                transforms.Compose([
                    transforms.Resize((64, 128)),
                    # transforms.RandomCrop((60, 120)),
                    # transforms.ColorJitter(0.3, 0.3, 0.2),
                    # transforms.RandomRotation(5),
                    transforms.ToTensor(),
                    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])),
    batch_size=40,
    shuffle=False,
)

40000 40000


In [18]:
# 加载保存的最优模型
model.load_state_dict(torch.load('./output/model.pt'))

test_predict_label = predict(test_loader, model, 1)
print(test_predict_label.shape)

test_label = [''.join(map(str, x)) for x in test_loader.dataset.img_label]
test_predict_label = np.vstack([
    test_predict_label[:, :11].argmax(1),
    test_predict_label[:, 11:22].argmax(1),
    test_predict_label[:, 22:33].argmax(1),
    test_predict_label[:, 33:44].argmax(1),
    test_predict_label[:, 44:55].argmax(1),
]).T

test_label_pred = []
for x in test_predict_label:
    test_label_pred.append(''.join(map(str, x[x!=10])))

import pandas as pd
df_submit = pd.read_csv('./input/mchar_sample_submit_A.csv')
df_submit['file_code'] = test_label_pred
df_submit.to_csv('./output/submit.csv', index=None)

(40000, 55)
